# Quickstart Dashboard 入门指南

本笔记帮助你快速体验 `notebooks/quickstart_dashboard.py` 中提供的轻量级统计仪表盘框架。
通过少量代码即可把 Git2Base 的分析结果渲染成可交互的项目/运行选择页面，并在单次执行与趋势视图中展示自定义指标。

## 1. 环境准备

确保你已经在 `artifacts/` 目录中生成了至少一次分析运行的数据。
"
    "目录结构应类似：
"
    "```
artifacts/
  your-project/
    runs/
      20240101-120000/
        data/
          commit_files.csv
          diff_results.csv
          ...
```
"
    "这些 CSV 文件会被仪表盘加载并转换成 Pandas DataFrame。

## 2. 引入仪表盘框架

In [ ]:
from quickstart_dashboard import QuickstartDashboard, RecordCountStatistic
from quickstart_dashboard import RunDataLoader  # 可选：自定义数据目录


## 3. 创建仪表盘实例
默认情况下，框架会从 `../artifacts` 读取数据。
如果你的数据位于其他位置，可以使用 `RunDataLoader` 指定新的路径。

In [ ]:
# 将 base_dir 替换成你实际存放分析结果的目录（相对于当前 notebook）。
dashboard = QuickstartDashboard(base_dir="../artifacts")
# 你也可以显式传入 loader：
# loader = RunDataLoader(base_dir="/path/to/artifacts")
# dashboard = QuickstartDashboard(loader=loader)


## 4. 注册基础统计
`RecordCountStatistic` 是一个简单示例：它会统计某个 CSV 表的行数。
可以快速验证仪表盘是否正常工作。

In [ ]:
dashboard.register_statistic(
    RecordCountStatistic(
        name="分析文件数",
        table_key="commit_files_df",
        description="每次运行处理的文件数量",
    )
)


## 5. 自定义你的统计卡片
通过继承框架中的 `BaseStatistic`（`RecordCountStatistic` 就是实现之一）即可新增统计。
下面演示如何统计每次运行涉及的唯一作者数量。

In [ ]:
from typing import Optional

import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt

from quickstart_dashboard import BaseStatistic, RunData, RunHistory, _stat_card


class UniqueAuthorsStatistic(BaseStatistic):
    """统计 diff 结果中出现的唯一作者数量。"""

    def __init__(self) -> None:
        self.name = "唯一作者数"
        self.description = "根据 diff_results.csv 的 author 列计算"

    def _extract(self, run: RunData) -> Optional[int]:
        df = run.dataframes.get("diff_results_df")
        if df is None or "author" not in df.columns:
            return None
        return int(pd.Series(df["author"]).dropna().nunique())

    def render_single(self, run: RunData):
        value = self._extract(run)
        display_value = value if value is not None else "N/A"
        return _stat_card(self.name, display_value, self.description)

    def render_trend(self, history: RunHistory):
        records = []
        for run in history.runs:
            value = self._extract(run)
            if value is not None:
                records.append({"label": run.display_label, "value": value})

        if not records:
            return widgets.HTML(value="<div style='color:#666;'>暂无唯一作者统计数据。</div>")

        df = pd.DataFrame(records)
        output = widgets.Output()
        with output:
            fig, ax = plt.subplots(figsize=(6, 3))
            ax.plot(df["label"], df["value"], marker="o")
            ax.set_title(self.name)
            ax.set_xlabel("Run")
            ax.set_ylabel("Authors")
            ax.grid(True, linestyle="--", alpha=0.3)
            plt.xticks(rotation=30, ha="right")
            plt.tight_layout()
            display(fig)
            plt.close(fig)

        header = widgets.HTML(
            value=f"<b>{self.name}</b><div style='color:#666;font-size:11px;'>{self.description}</div>"
        )
        return widgets.VBox([header, output])


In [ ]:
# 注册自定义统计（可按需添加多个）
dashboard.register_statistic(UniqueAuthorsStatistic())


## 6. 展示仪表盘
运行下方单元格，即可在 Jupyter 中看到交互式仪表盘。
当你新增或删除运行结果后，可调用 `dashboard.refresh()` 重新加载数据。

In [ ]:
dashboard.show()
# 如需刷新数据：
# dashboard.refresh()
